In [143]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [144]:
options = Options()

options.add_argument('--user-data-dir=/Users/Arian_Gh/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 4')

options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("window-size=1280,800")
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Add a preference to disable image loading
prefs = {
    "profile.managed_default_content_settings.images": 2
}
options.add_experimental_option("prefs", prefs)

capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True


driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)

In [133]:
html_text = driver.page_source
soup = BeautifulSoup(html_text,"lxml")

In [134]:
file_path = 'categories.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)

In [135]:
products = soup.find("div",class_='right-group flx1 df fww pr')
sections = products.find_all("section")
all_details = {}
index = 0
for section in sections:
    details={}
    try:
        name = section.find("a",class_="prd-name").text
        price = section.find("span",class_="elps elps2 fs14 tac clr7").text
        url = section.find("a",class_="prd-name")["href"].split(" ")[0]
        details["name"]= name
        details["price"] = price
        details["url"] = url
        all_details[index] = details
        index+=1
    except:
        try:
            name = section.find("span",class_="noPdp prd-name").text
            price = section.find("span",class_="elps elps2 fs14 tac clr7").text
            url = None
            details["name"]= name
            details["price"] = price
            details["url"] = url
            all_details[index] = details
            index+=1
        except:
            continue
    


In [140]:
all_details = {}
index = 0
urls = []
while True:
    html_text = driver.page_source
    soup = BeautifulSoup(html_text,"lxml")

    products = soup.find("div",class_='right-group flx1 df fww pr')
    sections = products.find_all("section")

    for section in sections:
        details={}
        try:
            name = section.find("a",class_="prd-name").text
            price = section.find("span",class_="elps elps2 fs14 tac clr7").text
            url = section.find("a",class_="prd-name")["href"].split(" ")[0]
            if url in urls:
                continue
            
            details["name"]= name
            details["price"] = price
            details["url"] = url
            urls.append(url)
            all_details[index] = details
            index+=1
        except:
            try:
                name = section.find("span",class_="noPdp prd-name").text
                price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                url = None
                details["name"]= name
                details["price"] = price
                details["url"] = url
                all_details[index] = details
                index+=1
            except:
                continue
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Give the page some time to load after scrolling (adjust the time as needed)
    driver.implicitly_wait(2)

    # Try to find the "load more" button and click it
    try:
        btn = driver.find_element(By.XPATH, "//*[@id='app']/div/main/div[4]/div[2]/div[2]/span")
        if btn.is_displayed(): # Check if the button is displayed
            btn.click()
    except NoSuchElementException:
        break

In [142]:
all_details

{0: {'name': 'Inconel 690 Scrap',
  'price': '₹550/ Kilogram',
  'url': 'https://www.indiamart.com/proddetail/inconel-690-scrap-11643041197.html?pos=1&kwd=industrial'},
 1: {'name': 'Titanium Scrap',
  'price': '₹250/ Kilogram',
  'url': 'https://www.indiamart.com/proddetail/titanium-scrap-11674300391.html?pos=3&kwd=industrial'},
 2: {'name': 'Stainless Steel Plate Cut Scrap, 50 KG, Packaging Type: GUNNY BAGS',
  'price': '₹100/ Kg',
  'url': 'https://www.indiamart.com/proddetail/stainless-steel-plate-cut-scrap-19281601388.html?pos=4&kwd=industrial'},
 3: {'name': 'Black Mild Steel Scrap, For Metal Industry, Packaging Type: Loose',
  'price': '₹43/ Kg',
  'url': 'https://www.indiamart.com/proddetail/mild-steel-scrap-26010043330.html?pos=5&kwd=industrial'},
 4: {'name': 'Aluminium Scrap Lithium Battery, Model Name/Number: 18650/2100/32650, Battery Capacity: 500 mAh',
  'price': '₹90/ Kg',
  'url': 'https://www.indiamart.com/proddetail/scrap-lithium-battery-24229959212.html?pos=6&kwd=ind